In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm import tqdm
import json

In [20]:
cities_og = pd.read_csv("worldcities.csv")
cities_og["area"] = None
cities_og

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id,area
0,Tokyo,Tokyo,35.6897,139.6922,Japan,JP,JPN,Tōkyō,primary,37977000.0,1392685764,None
1,Jakarta,Jakarta,-6.2146,106.8451,Indonesia,ID,IDN,Jakarta,primary,34540000.0,1360771077,None
2,Delhi,Delhi,28.6600,77.2300,India,IN,IND,Delhi,admin,29617000.0,1356872604,None
3,Mumbai,Mumbai,18.9667,72.8333,India,IN,IND,Mahārāshtra,admin,23355000.0,1356226629,None
4,Manila,Manila,14.5958,120.9772,Philippines,PH,PHL,Manila,primary,23088000.0,1608618140,None
...,...,...,...,...,...,...,...,...,...,...,...,...
26564,Nord,Nord,81.7166,-17.8000,Greenland,GL,GRL,Sermersooq,NaN,10.0,1304217709,None
26565,Timmiarmiut,Timmiarmiut,62.5333,-42.2167,Greenland,GL,GRL,Kujalleq,NaN,10.0,1304206491,None
26566,Cheremoshna,Cheremoshna,51.3894,30.0989,Ukraine,UA,UKR,Kyyivs’ka Oblast’,NaN,0.0,1804043438,None
26567,Ambarchik,Ambarchik,69.6510,162.3336,Russia,RU,RUS,Sakha (Yakutiya),NaN,0.0,1643739159,None


In [25]:
wiki_base_url = "https://en.wikipedia.org/wiki/"
cities_og["area"] = [None]*len(cities)
cities = cities_og.copy()
cities

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id,area
0,Tokyo,Tokyo,35.6897,139.6922,Japan,JP,JPN,Tōkyō,primary,37977000.0,1392685764,None
1,Jakarta,Jakarta,-6.2146,106.8451,Indonesia,ID,IDN,Jakarta,primary,34540000.0,1360771077,None
2,Delhi,Delhi,28.6600,77.2300,India,IN,IND,Delhi,admin,29617000.0,1356872604,None
3,Mumbai,Mumbai,18.9667,72.8333,India,IN,IND,Mahārāshtra,admin,23355000.0,1356226629,None
4,Manila,Manila,14.5958,120.9772,Philippines,PH,PHL,Manila,primary,23088000.0,1608618140,None
...,...,...,...,...,...,...,...,...,...,...,...,...
26564,Nord,Nord,81.7166,-17.8000,Greenland,GL,GRL,Sermersooq,NaN,10.0,1304217709,None
26565,Timmiarmiut,Timmiarmiut,62.5333,-42.2167,Greenland,GL,GRL,Kujalleq,NaN,10.0,1304206491,None
26566,Cheremoshna,Cheremoshna,51.3894,30.0989,Ukraine,UA,UKR,Kyyivs’ka Oblast’,NaN,0.0,1804043438,None
26567,Ambarchik,Ambarchik,69.6510,162.3336,Russia,RU,RUS,Sakha (Yakutiya),NaN,0.0,1643739159,None


### Crawling the Areas

1. Problem: The box on the right city of each of these city Wikipedia articles is a html table and contains rows and areas. The problem here is, that there are sometimes multiple entries for the area, denoting different area properties. In order to solve this problem, we will simply take the largest of these entries.

In [ ]:

# go through every city
for df_indx in tqdm(range(len(cities))):
    #print(df_indx, len(areas))
    if cities["area"].iloc[df_indx] == None:
        # get the html
        page = requests.get(wiki_base_url + cities.iloc[df_indx]["city"].replace(" ", "_"))

        # transform to soup object
        soup = BeautifulSoup(page.content, 'html.parser')

        # find the table with all information (right, top of wikipedia article)
        table = soup.find('table', class_='infobox geography vcard')


        # if no table was found, something wrent wong and we insert None and deal with it later
        if not table:
            print("Not Found: ", cities.iloc[df_indx]["city"])
            continue
        else:
            # for saving all tale entries of one webpage that containg km²
            temporary_list = []

            # get all rows of the table on the right of the wikipedia article
            rows = table.find_all("tr")

            # go over all rows
            for indx, row in enumerate(rows):
                # get all the cells withing the rows
                cells = row.find_all('td')
                # go through each cell
                for cell in cells:
                    # finding all km² entries and putting them into the temporary list, together with 
                    # the index of the corresponding row
                    # also delete the comma that separates thousand from hundreds
                    if "km2" in cell.text:
                        temporary_list.append((indx, float(re.sub("[^0-9.]", "", re.findall(r"\d{1,6}\.?\d{0,2}", cell.text.replace(",", ""))[0]))))
                        if not float(re.sub("[^0-9.]", "", re.findall(r"\d{1,6}\.?\d{0,2}", cell.text.replace(",", ""))[0])):
                            print("problem", cities.iloc[df_indx]["city"], ":", cell.text)

            temporary_list_2 = []
            found_indx = False
            for indx, cell_v in temporary_list:

                if not found_indx:
                    found_indx = indx
                    temporary_list_2.append(cell_v)
                else:
                    if indx != found_indx + 1:
                        break
                    else:
                        temporary_list_2.append(cell_v)
                        found_indx += 1
            if not temporary_list_2:
                print("Not Found2: ", cities.iloc[df_indx]["city"])
            else:
                cities["area"].iloc[df_indx] = max(temporary_list_2)
    
    if df_indx % 100 == 0:
        cities.to_csv("./city_and_area.csv")

In [12]:
len(areas)

21406

In [57]:
cities[cities["area"].isna()]["population"].sum()/cities["population"].sum()

0.2542301107318778

In [52]:
pd.read_csv("city_and_area.csv")

,Unnamed: 0,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id,area
0,0,Tokyo,Tokyo,35.6897,139.6922,Japan,JP,JPN,Tōkyō,primary,37977000.0,1392685764,2194.07
1,1,Jakarta,Jakarta,-6.2146,106.8451,Indonesia,ID,IDN,Jakarta,primary,34540000.0,1360771077,7062.50
2,2,Delhi,Delhi,28.6600,77.2300,India,IN,IND,Delhi,admin,29617000.0,1356872604,1484.00
3,3,Mumbai,Mumbai,18.9667,72.8333,India,IN,IND,Mahārāshtra,admin,23355000.0,1356226629,4355.00
4,4,Manila,Manila,14.5958,120.9772,Philippines,PH,PHL,Manila,primary,23088000.0,1608618140,1474.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26564,26564,Nord,Nord,81.7166,-17.8000,Greenland,GL,GRL,Sermersooq,NaN,10.0,1304217709,NaN
26565,26565,Timmiarmiut,Timmiarmiut,62.5333,-42.2167,Greenland,GL,GRL,Kujalleq,NaN,10.0,1304206491,NaN
26566,26566,Cheremoshna,Cheremoshna,51.3894,30.0989,Ukraine,UA,UKR,Kyyivs’ka Oblast’,NaN,0.0,1804043438,NaN
26567,26567,Ambarchik,Ambarchik,69.6510,162.3336,Russia,RU,RUS,Sakha (Yakutiya),NaN,0.0,1643739159,NaN
